In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

init_notebook_mode(connected=True)
plotly.__version__

'3.10.0'

In [23]:
scraped = pd.concat([pd.read_pickle('../datasets/TweetsScrapedUsers_k1.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_k2.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_k3.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_m1.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_m2.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_m3.pkl')])
original = pd.concat([pd.read_pickle('../datasets/Users_K.pkl'),\
                      pd.read_pickle('../datasets/Users_M.pkl')])
users = original[original.id.isin(scraped.user_id)] 
users = users[pd.DataFrame(users.id.astype(str)).id.isin(tweets.user_id)]
users = users[users.id.isin(users.id.unique())]

In [29]:
tweets = pd.concat([pd.read_pickle('../datasets/TweetsDataset_K.pkl'),\
                    pd.read_pickle('../datasets/TweetsDataset_M.pkl')])

In [30]:
# Arreglar valor no timestamp
tweets['created_at'] = pd.to_datetime(tweets.created_at,errors='coerce')

In [35]:
#tweets = pd.read_pickle('../datasets/TweetsDataset_final.pkl')
#users = pd.read_pickle('../datasets/UsersDataset.pkl')
print(tweets.columns)
display(users.head(3))
tweets.head()

Index(['tweet_id', 'user_id', 'screen_name', 'complete_name', 'text',
       'hashtags', 'mencioned_user_id', 'mencioned_screen_name', 'is_reply',
       'reply_screen_name', 'reply_user_id', 'all_reply_screen_name',
       'all_reply_user_id', 'is_retweet', 'retweet_user_id',
       'retweet_screen_name', 'retweet_complete_name', 'retweet_count',
       'reply_count', 'favorite_count', 'num_hashtags', 'num_urls',
       'num_mentions', 'created_at', 'crawled_at'],
      dtype='object')


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
347513688,347513688,cinedelabestia,El cine de La Bestia,14046,Cine y televisión bestiales.,Buenos Aires,False,False,0,2011-08-01 00:00:00,2019-06-29 23:15:54.650537,21379,2368,543,False
470238248,470238248,14capo,HUGO,32185,Hacia la Victoria Siempre ......,Por Ahí,False,False,0,2012-01-01 00:00:00,2019-07-12 23:36:30.289443,16309,2511,3996,False
4449919888,4449919888,ditulliojuli,Juliana di Tullio,11739,Diputada Nacional del Frente para la Victoria ...,False,22 de octubre,facebook.com/ditullioj,0,2015-12-01 00:00:00,2019-07-12 23:36:36.413663,13994,105598,1128,True


,tweet_id,user_id,screen_name,complete_name,text,hashtags,mencioned_user_id,mencioned_screen_name,is_reply,reply_screen_name,...,retweet_screen_name,retweet_complete_name,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,crawled_at
1150171442148982785,1150171442148982785,187920918,fsusbielles,Federico Susbielles,Visitamos la Sociedad de Fomento de Stella Mar...,[],[],[],False,NaN,...,NaN,NaN,11,2,21,0,1,0,2019-07-13 22:33:35,2019-07-14 01:39:21.823707
1147993842379034624,1147993842379034624,871232496763117568,CalvetDavid73,David Calvet,Que asco es la @CONMEBOL.\n\nPrimero manipulan...,"[PeruvsBrasil, BuenDomingo, CorrupcionConmebol]","[1265260098, 128372940]","[CONMEBOL, jairbolsonaro]",False,NaN,...,NaN,NaN,0,0,2,3,0,2,2019-07-07 22:20:35,2019-07-14 01:39:22.963764
1150098985413218305,1150098985413218305,187920918,fsusbielles,Federico Susbielles,Compartimos la mañana de sábado con vecinos y ...,[UnidosPorBahia],[],[],False,NaN,...,NaN,NaN,15,2,37,1,1,0,2019-07-13 17:45:40,2019-07-14 01:39:42.598669
1146627567618809857,1146627567618809857,871232496763117568,CalvetDavid73,David Calvet,pic.twitter.com/Q0DTsFIYQC,[],[],[],True,CalvetDavid73,...,NaN,NaN,1,0,3,0,1,0,2019-07-04 03:51:30,2019-07-14 01:39:42.911937
1150090047108124678,1150090047108124678,1369385587,graba829,angie fueyo,Allí estaremos @MaxDelupi como todos los domin...,[MacriEsPobreza],[7379612],[MaxDelupi],False,NaN,...,NaN,NaN,0,0,2,1,1,1,2019-07-13 17:10:09,2019-07-14 05:25:51.632502


In [36]:
#tweets_humans = tweets[tweets.Cat=='Humans']
#display(tweets_humans.columns)

#first = True
def getTimes(times):
    if(len(times) == 0):
        return np.array([0,0,0,0])
    return np.array([times.mean().seconds, times.median().seconds, times.std().seconds, times.quantile(0.25).seconds, times.quantile(0.75).seconds])

def getDist(var):
    if(len(var) == 0):
        return np.array([0,0,0,0])
    return np.array([var.mean(), var.median(), var.std(), var.quantile(0.25), var.quantile(0.75)])

def getWeekAv(tw):
    return len(tw) / len(tw.weekNum.unique())

def getDataTweets(twU):
    userId = twU.user_id.values[0]
    twU = twU.sort_values(by='created_at', ascending=0).fillna(0) #tweets del usuario
    dates = twU.created_at #aca lo que hago es guardar la columnoa como datetime

    twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
    twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
    twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))

    tweetsUser = twU[twU.is_retweet==False] #tweets propios del usuario
    tweetsRT = twU[twU.is_retweet==True] #retweets

#     twU.loc[:,'interval'] = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp)
    twU.loc[:,'interval'] = pd.to_datetime(twU.created_at.shift(1)) - pd.to_datetime(twU.created_at)
    times = pd.to_datetime(twU.created_at.shift(1)) - pd.to_datetime(twU.created_at) # calculo distancia entre tweets

    tweetsUser.loc[:,'interval'] = pd.to_datetime(tweetsUser.created_at.shift(1)) - pd.to_datetime(tweetsUser.created_at)
    timesUser = pd.to_datetime(tweetsUser.created_at.shift(1)) - pd.to_datetime(tweetsUser.created_at) # calculo distancia entre tweets

    tweetsRT.loc[:,'interval'] = pd.to_datetime(tweetsRT.created_at.shift(1)) - pd.to_datetime(tweetsRT.created_at)
    timesRT = pd.to_datetime(tweetsRT.created_at.shift(1)) - pd.to_datetime(tweetsRT.created_at) # calculo distancia entre tweets
    
    mediaSemanas = np.around(twU.groupby('dayOfWeek').apply(getWeekAv), 6)
    
    #ver word_to_vect
    result = np.concatenate([
        [
            userId,
            twU.tweet_id.count(),
            len(twU.weekNum.unique())
        ],
        mediaSemanas,
        getTimes(times),
        getTimes(timesRT),
        getTimes(timesUser),
        getDist(tweetsUser.favorite_count.astype(int)) * 10000,
        getDist(tweetsUser.retweet_count.astype(int)) * 10000,
        getDist(tweetsUser.num_mentions.astype(int)) * 10000,
        getDist(tweetsUser.reply_count.astype(int)) * 10000,

    ])
    
    return result

res = tweets.groupby('user_id').apply(getDataTweets)
cols = np.concatenate(
    [
        
        [x+'_mean',x+'_median', x+'_std', x+'_fq', x+'_tq'] for x in
        ['times','timesRT','timesUser','favorite_count','retweet_count','num_mentions','reply_count']
    ]
)
cols = np.concatenate([
    ['user_id', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom'],cols
])
res = pd.DataFrame(
    [x for x in res], columns = cols
)
res.index = res.user_id

print(len(res))

789


In [37]:
res.to_pickle('../datasets/tweetsProcesados_KM.pkl')